In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from data_fns import get_yelp_stars
from eval_utils import predict_stuff
import numpy as np

In [4]:
from eval_utils import load_model

path_name = '/u/scr/smurty/LanguageExplanations/gpt2-finetune-generalization/models/t5-large-sst-no-exp'
model_obj = load_model(path_name, primary_mode='exp_applies_predictor')


Some weights of T5ForConditionalGenerationMultipleHeads were not initialized from the model checkpoint at t5-large and are newly initialized: ['encoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


primary mode: exp_applies_predictor
splicing parts from pretrained model


Some weights of T5ForConditionalGeneration were not initialized from the model checkpoint at t5-large and are newly initialized: ['encoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


only loading base model!


Some weights of T5ForConditionalGeneration were not initialized from the model checkpoint at t5-large and are newly initialized: ['encoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


primary mode: exp_applies_predictor


In [5]:
tests_yelp = get_yelp_stars()
inps, labels = tests_yelp[0], tests_yelp[1]

Reusing dataset yelp_polarity (/u/scr/smurty/yelp_polarity/plain_text/1.0.0/a770787b2526bdcbfc29ac2d9beb8e820fbc15a03afd3ebc4fb9d8529de57544)


In [6]:
no_exps = [('', ex) for ex in inps]
no_exp_probs = predict_stuff(no_exps, [0]*len(no_exps), model_obj, 'p1', verbose=False, mode='task_predictor')
no_exp_preds = no_exp_probs.argmax(axis=1)

print(np.mean(no_exp_preds == labels))



  0%|          | 0/4 [00:00<?, ?ba/s]

0.9312736443883984


In [9]:
def rule_based_patch(inp, orig_pred):
    keywords = [' 0 star', ' 1 star', ' 2 star', 
               ' zero star', ' one star', ' two star']
    if any(keyword in inp for keyword in keywords):
        return 0
    else:
        return orig_pred

rule_preds = np.array([rule_based_patch(inp, pred) for inp, pred in zip(inps, no_exp_preds)])
print(np.mean(rule_preds == labels))



0.919609079445145


In [8]:
prompt ='If review gives 0, 1, 2 stars, then label is negative'
import itertools
def get_prompting(data, prompt, examine=False):
    prompted_inps = [(prompt, ex) for ex in data[0]]
    probs = predict_stuff(prompted_inps, itertools.repeat(0), 
                          model_obj, 'p1', verbose=False, mode='task_predictor')
    #print(np.mean(probs.argmax(axis=1)==data[1]))
    return probs.argmax(axis=1)
    


prompt_preds = get_prompting((inps, labels), prompt, examine=False)
print(np.mean(prompt_preds == labels))        



  0%|          | 0/4 [00:00<?, ?ba/s]

0.9082597730138714


In [10]:
import pickle
with open('wcr.pickle', 'rb') as reader:
    dataset = pickle.load(reader)

In [11]:
explanations = [('review says fit is boxy',[1,0]),
               ("review contains words or phrases like needs to be returned", [1, 0]),
               ("review contains words or phrases like needs to be exchanged", [1,0])]
    

no_exps = [('', data) for data in dataset[0]]
baseline_rec_out = predict_stuff(no_exps, [0]*len(no_exps), model_obj, 'p1', verbose=True, mode='task_predictor')
base_preds = baseline_rec_out.argmax(axis=1)

print(np.mean(base_preds == dataset[1]))

("Input: ...the print is so sharking, and i love the way it looks on the model -- but i'm a more curvy figure, and the boxy-ish cut plus rather stuff fabric in front is incredibly unflattering. ordinarily i love everything made by maeve, but this one sadly must be returned... on a thinner/straighter-shaped person i expect it would be great", 'negative')


  0%|          | 0/3 [00:00<?, ?ba/s]

100%|██████████| 46/46 [00:14<00:00,  3.26it/s]

0.8958547447756081


In [12]:
def rule_based_patch2(inp, orig_pred):
    keywords = ['boxy', 
               'needs to be returned']
    if any(keyword in inp for keyword in keywords):
        return 0
    else:
        return orig_pred
    
rule_preds = np.array([rule_based_patch2(inp, orig_pred) for inp, orig_pred in zip(dataset[0], base_preds)])
print(np.mean(rule_preds == dataset[1]))

0.8999657416923604
